# Library

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from datasets import load_dataset, Dataset
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)

from trl.core import LengthSampler
from trl import (
    PPOTrainer,
    PPOConfig,
    AutoModelForCausalLMWithValueHead,
    create_reference_model,
    DPOConfig,
    DPOTrainer,
)

# Reinforcement Learning with Human Feedback

paper: https://arxiv.org/pdf/2203.02155

![](https://huyenchip.com/assets/pics/rlhf/6-sft-rlhf.png)

<br>

<font style="font-size:20px"> 기존 모델의 한계 </font>

**신뢰할 수 없는 답변이나, 유해하거나, 유저에게 도움이 안 되는 결과를 생성**

## RLHF

사람의 선호를 reward로 하여 모델을 fine-tuning

<br>

<font style="font-size:20px"> 학습 방법 </font>

1. 레이블러는 입력 프롬프트에서 원하는 동작의 데모 제공. <br>
지도 학습을 사용하여 이 데이터에서 사전 학습된 GPT-3 모델 fine-tuning.

2. 주어진 입력에서 레이블러가 선호하는 output 사이의 비교 데이터 수집. <br>
reward model을 학습시켜 선호하는 출력 예측

3. PPO를 사용하여 reward model에 대한 policy 최적화.

### Model

1. Supervised Fine-Tuning (SFT): <br>
레이블된 데이터로 GPT-3 모델을 미세 조정

2. Reward Modeling (RM): <br>
SFT 모델에서 prompt와 response를 기반으로 reward를 출력하도록 모델 학습
방법:

3. Reinforcement Learning (RL): <br>
PPO를 사통해 SFT 모델을 미세 조정하는 것.

## Practice

### 긍정 답변 유도

In [ ]:
# GPT-2 모델용 사전 학습된 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained('lvwerra/gpt2-imdb')

# 패딩 토큰을 설정합니다. 여기서는 EOS(End Of Sentence) 토큰을 패딩 토큰으로 지정합니다.
tokenizer.pad_token = tokenizer.eos_token

# IMDb 데이터셋의 'train' split을 로드합니다.
data = load_dataset('imdb', split='train')

# 데이터셋의 'text' 열 이름을 'review'로 변경하여 일관성을 유지합니다.
data = data.rename_columns({'text': 'review'})

# 각 리뷰 텍스트 길이가 200자 이상인 샘플만 필터링합니다.
data = data.filter(lambda x: len(x['review']) > 200)

# 데이터셋의 텐서 형식을 'pt' (PyTorch 텐서)로 설정하여 모델에 바로 사용할 수 있도록 합니다.
data.set_format('pt')

In [ ]:
# 입력 텍스트의 최소 및 최대 길이를 정의합니다.
input_min_text_length = 2    # 최소 텍스트 길이
input_max_text_length = 8    # 최대 텍스트 길이

# LengthSampler 객체를 사용하여, 입력 텍스트 길이를 최소와 최대 길이 범위 내에서 무작위로 샘플링합니다.
input_size = LengthSampler(input_min_text_length, input_max_text_length)

In [ ]:
# IMDb 데이터셋의 각 샘플에 대해 토큰화 및 변환을 수행하는 함수입니다.
def tokenize(sample):
    # 'review' 텍스트를 토크나이즈하고, 지정된 입력 길이(input_size)로 자릅니다.
    sample['input_ids'] = tokenizer.encode(sample['review'])[: input_size()]
    
    # 토큰화된 'input_ids'를 다시 텍스트로 디코딩하여 'query' 키에 저장합니다.
    sample['query'] = tokenizer.decode(sample['input_ids'])
    
    # 토큰화가 완료된 샘플을 반환합니다.
    return sample

In [ ]:
# IMDb 데이터셋의 각 샘플에 대해 'tokenize' 함수를 적용하여 텍스트를 토큰화하고 필요한 필드를 추가합니다.
data = data.map(tokenize)

In [ ]:
# 데이터 로더의 배치 단위로 데이터를 구성하는 collator 함수입니다.
def collator(data):
    # 데이터 리스트(data)에서 동일한 키의 값을 모아 배치를 구성하는 딕셔너리를 반환합니다.
    # 각 키에 대해 해당 키를 가진 모든 샘플의 값을 리스트로 묶습니다.
    return dict((key, [datum[key] for datum in data]) for key in data[0])

In [ ]:
# 'lvwerra/gpt2-imdb'로 사전 학습된 Causal Language Model(자연어 생성 모델)을 로드합니다.
# 강화 학습(RL)을 위해 value head를 추가하여 감정이나 보상 등 특정 값 예측이 가능하도록 합니다.
model = AutoModelForCausalLMWithValueHead.from_pretrained('lvwerra/gpt2-imdb')

# 학습 전 원래 사전 학습된 모델을 참조 모델로 로드합니다.
# 이는 학습 후 모델이 원래 모델과 얼마나 다른지 비교하거나 평가할 때 사용됩니다.
model_reference = AutoModelForCausalLMWithValueHead.from_pretrained('lvwerra/gpt2-imdb')

In [ ]:
# Proximal Policy Optimization(PPO) 구성 설정을 정의합니다.
config = PPOConfig(
    model_name='lvwerra/gpt2-imdb',  # 사용할 모델 이름
    learning_rate=2e-5,              # 학습률
    log_with='wandb',                # 실험 추적 도구로 Weights and Biases를 사용하도록 지정
)

# PPO 알고리즘을 사용하여 모델을 학습시키기 위한 트레이너(PPOTrainer)를 초기화합니다.
ppo_trainer = PPOTrainer(
    config,               # PPO 구성 설정
    model,                # 학습할 모델
    model_reference,      # 참조 모델 (기존 사전 학습된 모델)
    tokenizer,            # 토크나이저
    dataset=data,         # 데이터셋 (IMDb 데이터셋을 전처리한 것)
    data_collator=collator,  # 배치 구성을 위한 collator 함수
)

In [ ]:
# Hugging Face의 파이프라인을 사용하여 감정 분석 모델을 초기화합니다.
model_reward = pipeline(
    'sentiment-analysis',  # 감정 분석 태스크 지정
    model='lvwerra/distilbert-imdb',  # 사용할 사전 학습된 DistilBERT 모델 지정
)

In [ ]:
# 'bert-base-uncased'로 사전 학습된 BERT 모델을 로드하여 텍스트 분류 작업을 위한 모델을 초기화합니다.
bert = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

In [ ]:
# 'this move is horrible'라는 텍스트에 대한 감정 분석을 수행합니다.
# top_k 파라미터를 None으로 설정하여 모델이 모든 감정 클래스의 점수를 반환하도록 합니다.
result = model_reward('this move is horrible', top_k=None)

In [ ]:
# 출력 텍스트의 최소 및 최대 길이를 정의합니다.
output_min_length = 4    # 최소 출력 길이
output_max_length = 20   # 최대 출력 길이

# LengthSampler 객체를 사용하여, 출력 텍스트 길이를 최소와 최대 길이 범위 내에서 무작위로 샘플링합니다.
output_length_sampler = LengthSampler(
    output_min_length,     # 출력 길이의 최소값
    output_max_length      # 출력 길이의 최대값
)

In [ ]:
# PPO 트레이너의 데이터 로더를 통해 각 에포크와 배치를 반복합니다.
for epoch, batch in enumerate(tqdm(ppo_trainer.dataloader)):
    # 현재 배치에서 입력 쿼리의 ID를 가져옵니다.
    queries = batch.get('input_ids')

    response_tensors = []  # 생성된 응답을 저장할 리스트

    # 각 쿼리에 대해 응답을 생성합니다.
    for query in queries:
        # 출력 길이를 샘플링합니다.
        generation_length = output_length_sampler()
        
        # 모델을 사용하여 응답을 생성합니다.
        response = ppo_trainer.generate(
            query,
            min_length=-1,           # 생성할 최소 길이
            top_k=0,                 # Top-k 샘플링 비활성화
            top_p=1,                 # Top-p 샘플링을 사용하여 무작위 생성
            do_sample=True,          # 샘플링 모드 활성화
            pad_token_id=tokenizer.eos_token_id,  # 패딩 토큰 ID 설정
            max_new_tokens=generation_length  # 생성할 최대 새로운 토큰 수
        )
        response = response[0]
        
        # 응답의 길이를 계산하여 응답 텐서를 저장합니다.
        response_length = len(response) - len(query)
        response_tensors.append(response[-response_length:])
    
    # 생성된 응답을 디코딩하여 배치에 추가합니다.
    batch['response'] = [tokenizer.decode(response.squeeze())
                         for response in response_tensors]
    
    # 쿼리와 응답을 합쳐서 텍스트 리스트를 생성합니다.
    texts = [''.join([query, response])
             for query, response
             in zip(batch.get('query'), batch.get('response'))]
    
    # 합쳐진 텍스트에 대한 감정 점수를 계산합니다.
    sentiment_scores = model_reward(
        texts,
        top_k=None,              # 모든 감정 클래스의 점수를 반환
        batch_size=32,          # 배치 크기 설정
    )
    
    # 긍정 점수를 추출하여 보상으로 사용합니다.
    positive_scores = [
        item.get('score')
        for sentiment_score in sentiment_scores
        for item in sentiment_score
        if item.get('label') == 'POSITIVE'
    ]
    
    # 보상을 텐서로 변환합니다.
    rewards = [torch.tensor(score) for score in positive_scores]

    # PPO 트레이너의 한 스텝을 진행하고 통계를 기록합니다.
    stats = ppo_trainer.step(queries, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

In [ ]:
def make_sentiment_form(text, model, tokenizer):
    # 입력 텍스트를 토크나이저를 사용하여 텐서 형태로 변환합니다.
    tokenized = tokenizer(text, return_tensors='pt')
    
    # 모델의 그래디언트 계산을 비활성화하여 메모리 사용을 최적화합니다.
    with torch.no_grad():
        # 토큰화된 입력을 모델에 전달하여 예측 결과를 얻습니다.
        outputs = model(**tokenized)
    
    # 모델의 로짓에 소프트맥스 함수를 적용하여 확률 분포를 계산합니다.
    probs = F.softmax(outputs.logits)[0]
    
    # 긍정 및 부정의 확률 점수를 기반으로 결과를 구성합니다.
    positive = {'label': 'POSITIVE', 'score': probs[1].item()}  # 긍정 점수
    negative = {'label': 'NEGATIVE', 'score': probs[0].item()}  # 부정 점수
    
    # 긍정 및 부정 결과를 리스트 형태로 반환합니다.
    return [positive, negative]

In [ ]:
# 사용할 BERT 모델과 토크나이저의 이름을 정의합니다.
model_name = 'bert-base-uncased'

# 사전 학습된 BERT 모델을 로드합니다. 이 모델은 텍스트 분류를 위한 것입니다.
bert_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 해당 모델에 맞는 토크나이저를 로드합니다.
bert_tokenizer = AutoTokenizer.from_pretrained(model_name)

# 감정 분석을 수행할 입력 텍스트를 정의합니다.
text = 'how\'s your days going on?'

# 정의한 텍스트에 대해 감정 분석을 수행하고 결과를 반환합니다.
sentiment_result = make_sentiment_form(text, bert_model, bert_tokenizer)

In [ ]:
## 위의 rlhf의 코드에서 model_reward를 직접 학습한 bert model로 변경
## data: imdb
## model: AutoModelForSequenceClassification

## 단계
## - 1: imdb data로 bert 모델 학습
## BERT 모델과 토크나이저를 초기화합니다.
model_name = 'google-bert/bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## IMDb 데이터셋의 각 샘플을 전처리하는 함수입니다.
def preprocessing(sample):
    return tokenizer(
        sample['text'],
        padding='max_length',
        max_length=256,
        truncation=True,
        return_tensors='pt'
    )

## 데이터셋에 전처리 함수를 적용하고 훈련 및 검증 데이터셋을 나눕니다.
data = load_dataset('imdb')
train = data.map(preprocessing, batched=True)
eval = train.select(range(20000, 25000)) # 평가 데이터셋: 20000~25000번째 샘플
train = train.select(range(20000))       # 훈련 데이터셋: 처음 20000개의 샘플

## 모델 훈련을 위한 하이퍼파라미터를 설정합니다.
training_args = TrainingArguments(
    output_dir='./imdb',            # 모델과 로그 저장 경로
    eval_strategy='epoch',          # 매 에포크마다 평가를 수행
    learning_rate=2e-5,             # 학습률
    warmup_steps=50,                # 웜업 스텝 설정
    per_device_train_batch_size=16, # 훈련 배치 크기
    per_device_eval_batch_size=16,  # 평가 배치 크기
    num_train_epochs=2,             # 에포크 수
    weight_decay=0.01,              # 가중치 감소율
    logging_dir='./imdb',           # 로그 디렉토리
)

## Trainer를 초기화하여 BERT 모델을 IMDb 데이터셋으로 학습할 준비를 합니다.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,  # 훈련 데이터셋 설정
    eval_dataset=eval,    # 평가 데이터셋 설정
)

## 모델을 훈련합니다.
trainer.train()

## 훈련이 완료된 모델을 저장합니다.
model.save_pretrained('./imdb')

## - 2: 학습된 모델을 model_reward로 로드
##      - 학습된 모델을 'model_reward'로 로드하여, 이후 RLHF에서 보상 모델로 사용합니다.
model_reward = AutoModelForSequenceClassification.from_pretrained('./imdb')
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

## - 3: model의 결괏값을 아래의 형태로 변경
##      [{'label': label, 'score': score}, {'label': label, 'score': score}]
##      - positive: 긍정 예측 점수, negative: 부정 예측 점수
def make_sentiment_form(text, model, tokenizer):
    tokenized = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**tokenized)
    probs = F.softmax(outputs.logits)[0]
    
    positive = {'label': 'POSITIVE', 'score': probs[1].item()}
    negative = {'label': 'NEGATVE', 'score': probs[0].item()}
    
    return [positive, negative]

## 4: RLHF 단계 - 학습을 위한 데이터 토크나이즈
input_min_text_length = 2
input_max_text_length = 8
input_size = LengthSampler(input_min_text_length, input_max_text_length)

## 샘플을 토크나이즈하고 쿼리 텍스트를 저장합니다.
def tokenize(sample):
    sample['input_ids'] = tokenizer.encode(sample['review'])[: input_size()]
    sample['query'] = tokenizer.decode(sample['input_ids'])

    return sample

data = data.map(tokenize)

## 데이터 배치를 위한 Collator를 설정합니다.
def collator(data):
    return dict((key, [datum[key] for datum in data]) for key in data[0])

## PPO 학습을 위해 모델, 참조 모델, 구성 등을 초기화합니다.
model = AutoModelForCausalLMWithValueHead.from_pretrained('lvwerra/gpt2-imdb')
model_reference = AutoModelForCausalLMWithValueHead.from_pretrained('lvwerra/gpt2-imdb')

config = PPOConfig(
    model_name='lvwerra/gpt2-imdb',
    learning_rate=2e-5,
    log_with='wandb',
)

ppo_trainer = PPOTrainer(
    config,
    model,
    model_reference,
    tokenizer,
    dataset=data,
    data_collator=collator,
)

## 생성 응답 길이를 설정하는 샘플러입니다.
output_min_length = 4
output_max_length = 20
output_length_sampler = LengthSampler(
    output_min_length,
    output_max_length,
)

## RLHF 학습 루프 - 각 에포크에 대해 보상을 계산하고, 모델을 업데이트합니다.
for epoch, batch in enumerate(tqdm(ppo_trainer.dataloader)):
    queries = batch.get('input_ids')

    ## 응답 생성 및 응답 텐서를 수집합니다.
    response_tensors = []
    for query in queries:
        generation_length = output_length_sampler()
        response = ppo_trainer.generate(
            query,
            min_length=-1,
            top_k=0,
            top_p=1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=generation_length
        )
        response = response[0]
        response_length = len(response) - len(query)
        response_tensors.append(response[-response_length:])
    batch['response'] = [tokenizer.decode(response.squeeze())
                         for response
                         in response_tensors]

    ## 쿼리와 응답 텍스트를 합쳐 감성 분석을 수행하고 긍정 점수를 추출합니다.
    texts = [''.join([query, response])
             for query, response
             in zip(batch.get('query'), batch.get('response'))]
    sentiment_scores = [
      make_sentiment_form(text, model_reward, bert_tokenizer)
      for text
      in texts
    ]
    positive_scores = [
        item.get('score')
        for sentiment_score in sentiment_scores
        for item in sentiment_score
        if item.get('label') == 'POSITIVE'
    ]

    ## 보상을 생성하여 PPO 트레이너의 학습 단계로 전달합니다.
    rewards = [torch.tensor(score) for score in positive_scores]

    ## 학습 통계를 기록하고 PPO 학습 단계를 실행합니다.
    stats = ppo_trainer.step(queries, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

### 새로운 reward 추가

## Direct Preference Optimization (DPO)

paper: https://arxiv.org/pdf/2305.18290

![](https://miro.medium.com/v2/resize:fit:1100/format:webp/0*ObYqymfwm0m2hkjM)

<br>


<font style="font-size:20px"> 기존 모델의 한계 </font>

**기존 preference dataset으로 Reward Model을 학습해야 함.** <br>
일반 유저가 고성능의 reward model을 학습하는 것에 제약 존재. <br>


### DPO

Reinforcement Learning objective를 간단한 binary cross entropy object로 변환. <br>
-> RLHF를 위해 **reward model 없애고 objective를 단순화**

<br>

<font style="font-size:20px"> 학습 방법 </font>

RL Fine-Tuning Phase
$$
\max_{\pi_\theta} \mathbb{E}_{x\sim\mathcal{D},y\sim\pi_\theta(y|x)}[r_\phi(x,y)] - \beta\mathbb{D}_\text{KL}[\pi_\theta(y|x)\|\pi_\text{ref}(y|x)]
$$

-> Deriving the DPO object

$$
\pi_r(y|x) = \frac{1}{Z(x)}\pi_\text{ref}(y|x)\exp\left(\frac{1}{\beta}r(x,y)\right)
$$

### 사용 방법

> ```python
> # 1. dataset 객체 형성 (datasets의 dataset)
> # 반드시 prompt, chosen, rejected의 key값을 가지고 있어야 함
> 
> # 2. DPOConfig 설정
> model_name = ''
> training_args = DPOConfig(
>     output_dir=<output_dir>,
>     per_device_train_batch_size=16,
>     per_device_eval_batch_size=16,
>     logging_steps=10,
>     num_train_epochs=2,
>     )
> 
> # 3. 모델 로드
> model = AutoModelForCausalLM.from_pretrained(<model_repo_id>)
> tokenizer = AutoTokenizer.from_pretrained(<model_repo_id>)
> # tokenizer에서 pad_token 에러가 나는 경우 아래 실행
> tokenizer.pad_token = tokenizer.eos_token
> 
> # 4. Trainer 객체 정의
> trainer = DPOTrainer(
>     model=model,
>     args=training_args,
>     train_dataset=dataset,
>     tokenizer=tokenizer,
>   )
> 
> # 5. train
> trainer.train()
> ```

### Practice

#### Qwen

In [ ]:
# 사용할 모델의 이름을 지정합니다.
model_name = 'Qwen/Qwen2.5-0.5B-Instruct'

# 지정된 모델 이름을 기반으로 사전 학습된 언어 모델을 불러옵니다.
# AutoModelForCausalLM은 텍스트 생성 모델에 적합한 클래스입니다.
model = AutoModelForCausalLM.from_pretrained(model_name)

# 모델의 토크나이저를 불러옵니다. 토크나이저는 텍스트를 토큰으로 변환하거나 토큰을 텍스트로 변환하는 역할을 합니다.
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
# 주어진 샘플에서 필요한 텍스트를 추출하여 전처리합니다.
# 'prompt'는 'chosen'의 첫 번째 항목에서, 'chosen'과 'rejected'는 두 번째 항목에서 각각 'content' 값을 가져옵니다.
def preprocessing(sample):
    return {
        'prompt': sample.get('chosen')[0].get('content'),
        'chosen': sample.get('chosen')[1].get('content'),
        'rejected': sample.get('rejected')[1].get('content'),
    }

# 'trl-lib/ultrafeedback_binarized' 데이터셋의 'train' split을 로드합니다.
dataset = load_dataset('trl-lib/ultrafeedback_binarized', split='train')

# 각 샘플에 대해 preprocessing 함수를 적용하고, 'score_chosen' 및 'score_rejected' 열을 제거합니다.
dataset = dataset.map(
    preprocessing,
    remove_columns=['score_chosen', 'score_rejected'],
)

# 필터를 사용해 'chosen', 'rejected', 'prompt'의 길이가 모두 50자 이상인 샘플만 남깁니다.
dataset = dataset.filter(
    lambda sample: len(sample.get('chosen')) > 50
                    and len(sample.get('rejected')) > 50
                    and len(sample.get('prompt')) > 50
)

In [ ]:
# Direct Preference Optimization (DPO) 학습 설정을 구성합니다.
dpo_args = DPOConfig(
    output_dir='qwen',                       # 모델 학습 결과가 저장될 디렉터리입니다.
    per_device_train_batch_size=4,           # 훈련 시 각 디바이스(예: GPU) 당 배치 크기입니다.
    per_device_eval_batch_size=4,            # 평가 시 각 디바이스 당 배치 크기입니다.
    logging_steps=10,                        # 로그를 출력할 간격(스텝 수)입니다.
    num_train_epochs=2,                      # 전체 데이터셋에 대해 반복할 학습 에포크 수입니다.
)

# DPO를 사용해 학습할 Trainer를 설정합니다.
trainer = DPOTrainer(
    model=model,                             # 사전 학습된 언어 모델을 전달합니다.
    args=dpo_args,                           # 위에서 정의한 DPO 학습 설정을 전달합니다.
    train_dataset=dataset,                   # 훈련 데이터셋을 설정합니다.
    tokenizer=tokenizer,                     # 토크나이저를 설정합니다. 입력 데이터 처리를 돕습니다.
)

# 학습을 시작합니다.
trainer.train()

#### llama

In [ ]:
# 'argilla/distilabel-intel-orca-dpo-pairs' 데이터셋을 로드합니다.
dataset = load_dataset('argilla/distilabel-intel-orca-dpo-pairs')

# 훈련 데이터셋('train' split)을 선택합니다.
dataset = dataset['train']

# 사용할 열을 'input', 'chosen', 'rejected'로 제한합니다.
dataset = dataset.select_columns(['input', 'chosen', 'rejected'])

# 'input' 열의 이름을 'prompt'로 변경하여 모델 입력에 적합하도록 합니다.
dataset = dataset.rename_columns({'input': 'prompt'})

# 'chosen', 'rejected', 'prompt' 텍스트의 길이가 50자 이상인 샘플만 필터링합니다.
dataset = dataset.filter(
    lambda sample: len(sample.get('chosen')) > 50
                    and len(sample.get('rejected')) > 50
                    and len(sample.get('prompt')) > 50
)

# LLaMA 모델을 로드합니다.
model_name = 'meta-llama/Llama-3.2-1B-Instruct'
model = AutoModelForCausalLM.from_pretrained(model_name)

# 모델에 맞는 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 패딩 토큰을 엔드 토큰(eos_token)으로 설정하여 일관성 있게 입력을 패딩합니다.
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Direct Preference Optimization (DPO) 학습 설정을 구성합니다.
dpo_args = DPOConfig(
    output_dir='llama',                        # 모델 학습 결과를 저장할 디렉터리 경로입니다.
    per_device_train_batch_size=1,             # 훈련 시 디바이스(예: GPU) 당 배치 크기입니다.
    per_device_eval_batch_size=1,              # 평가 시 디바이스 당 배치 크기입니다.
    logging_steps=10,                          # 로그를 출력할 간격(스텝 수)입니다.
    num_train_epochs=2,                        # 전체 데이터셋에 대해 반복할 학습 에포크 수입니다.
)

# DPO를 사용해 학습할 DPOTrainer를 설정합니다.
trainer = DPOTrainer(
    model=model,                               # 사전 학습된 언어 모델을 전달합니다.
    args=dpo_args,                             # 학습 설정을 전달합니다.
    train_dataset=dataset,                     # 훈련 데이터셋을 설정합니다.
    tokenizer=tokenizer,                       # 입력 데이터를 처리하기 위한 토크나이저입니다.
)

# 학습을 시작합니다.
trainer.train()